In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
from src.scraper import RTScraper

In [11]:
scraper = RTScraper('https://russian.rt.com/rss')
news_df = scraper.get_latest_news()

2023-11-13 23:52:25.200 | INFO     | src.scraper:__get_news:62 - Length of news 50
/Users/semionkaminsky/opt/anaconda3/lib/python3.8/site-packages/bs4/__init__.py:221: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


In [12]:
news_df

,title,link,publication_time,text
0,Кержаков уволен с поста главного тренера «Спар...,https://russian.rt.com/sport/news/1230573-kerz...,2023-11-13 23:50:59+03:00,Россиянин Александр Кержаков уволен с поста гл...
1,Президент Польши поручил Моравецкому сформиров...,https://russian.rt.com/world/news/1230570-pols...,2023-11-13 23:50:05+03:00,Президент Польши Анджей Дуда поручил Матеушу М...
2,Спецоперация Z: хроника главных военных событи...,https://russian.rt.com/ussr/video/1230546-spec...,2023-11-13 23:49:32+03:00,Снайперы подразделений спецназа ВС РФ под Угле...
3,Березуцкий: конкурентная среда должна быть в Р...,https://russian.rt.com/sport/news/1230569-bere...,2023-11-13 23:43:17+03:00,Бывший защитник сборной России и ЦСКА Василий ...
4,«Зеркало недели» сообщает о взрывах в Харьковс...,https://russian.rt.com/ussr/news/1230568-harko...,2023-11-13 23:40:32+03:00,Взрывы произошли в Харьковской области Украины...
5,Израильские военные сфотографировались за триб...,https://russian.rt.com/world/news/1230567-parl...,2023-11-13 23:39:21+03:00,Израильские военные зашли в здание парламента ...
6,МИД: позиция США парализовала работу Совбеза О...,https://russian.rt.com/world/news/1230564-ssha...,2023-11-13 23:35:57+03:00,Совбез ООН не может выполнить свой прямой манд...
7,Хелвани: реванш Анкалаева и Уокера состоится в...,https://russian.rt.com/sport/news/1230565-reva...,2023-11-13 23:29:16+03:00,Российский боец Магомед Анкалаев проведёт рева...
8,«Не могли решить иначе»: в сборной России выст...,https://russian.rt.com/sport/article/1230299-g...,2023-11-13 23:21:34+03:00,Тренерский штаб сборной России по спортивной г...
9,Белый дом: Байден работает над продолжением ок...,https://russian.rt.com/world/news/1230562-bely...,2023-11-13 23:20:48+03:00,Администрация Джо Байдена активно работает с к...
